In [10]:
import pandas as pd
df = pd.read_csv('./data/final_dataset.csv')
df = df.drop(columns=['Unnamed: 0'])
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,1,6448,1,0.000388,0.000790,2,0,7,1,0,0,0
1,1,1376,1,0.000388,0.000426,-1,0,7,0,3,0,0
2,1,1883,1,0.000388,0.000822,4,0,7,2,30,0,1
3,1,6716,1,0.000388,0.005997,2,0,7,40,30,0,0
4,1,4710,1,0.000388,0.000421,-1,0,7,0,8,0,0


In [11]:
df.shape

(11366, 12)

In [12]:
# standarlize
x = df['Page_Rank_Src']
df['Page_Rank_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Page_Rank_Dst']
df['Page_Rank_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Shortest_Path']
df['Shortest_Path'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Dst']
df['Followees_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followees_Src']
df['Followees_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Dst']
df['Followers_Dst'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Followers_Src']
df['Followers_Src'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followees']
df['Int_Followees'] = ( x-x.min()) / ( x.max() - x.min() )
x = df['Int_Followers']
df['Int_Followers'] = ( x-x.min()) / ( x.max() - x.min() )
df.head(5)

,Source,Destination,Class,Page_Rank_Src,Page_Rank_Dst,Shortest_Path,Followers_Src,Followees_Src,Followers_Dst,Followees_Dst,Int_Followers,Int_Followees
0,1,6448,1,0.0,0.009239,0.428571,0.0,0.104478,0.010526,0.000000,0.0,0.000000
1,1,1376,1,0.0,0.000872,0.000000,0.0,0.104478,0.000000,0.044776,0.0,0.000000
2,1,1883,1,0.0,0.009970,0.714286,0.0,0.104478,0.021053,0.447761,0.0,0.045455
3,1,6716,1,0.0,0.128814,0.428571,0.0,0.104478,0.421053,0.447761,0.0,0.000000
4,1,4710,1,0.0,0.000757,0.000000,0.0,0.104478,0.000000,0.119403,0.0,0.000000


In [13]:
from sklearn.model_selection import train_test_split
df_x = df.drop(columns=['Source', 'Destination', 'Class'])
df_y = df[['Class']]
x_train, x_test, y_train, y_test  = train_test_split(df_x, df_y, test_size = 0.3, random_state=42)

In [14]:
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

estimators = [50, 100, 150]
depths = [9, 12, 15, 24]
split = np.random.randint(100, 150, 2)
leaves = np.random.randint(20, 30, 2)
param_grid = {'n_estimators' : estimators, 'max_depth' : depths, 'min_samples_split' : split, 'min_samples_leaf' : leaves}

gs_rf = GridSearchCV(cv = 5, estimator = RandomForestClassifier(), param_grid = param_grid)
gs_rf.fit(x_train, y_train.values.ravel())

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [9, 12, 15, 24],
                         'min_samples_leaf': array([24, 25]),
                         'min_samples_split': array([121, 119]),
                         'n_estimators': [50, 100, 150]})

In [17]:
import pickle

pickle.dump(gs_rf.best_estimator_, open('./data/rf_model.p', 'wb')) 
print(gs_rf.best_params_)

{'max_depth': 12, 'min_samples_leaf': 25, 'min_samples_split': 119, 'n_estimators': 100}


In [18]:
from sklearn.metrics import classification_report
print(classification_report(gs_rf.best_estimator_.predict(x_test), y_test)) 

              precision    recall  f1-score   support

           0       0.90      0.87      0.88      1731
           1       0.87      0.90      0.88      1679

    accuracy                           0.88      3410
   macro avg       0.88      0.88      0.88      3410
weighted avg       0.88      0.88      0.88      3410

